# Denver Metropolitan Area, Colorado, USA

I chose to work on the Open Street Map data for my hometown, Boulder, Colorado and it's surrounding area, including the state capital of Denver.

https://mapzen.com/data/metro-extracts/metro/denver-boulder_colorado/

## Data Auditing and Cleaning

The first field I looked at was the Street Name field, continuing with the code from one of the class exercises. From the audit, I created a full list of acceptable street names, as well as a mapping from abbreviations and other forms of the name that I saw in the data to these acceptable forms. Unlike in the exercise, I modified the code to apply the mapping to each word in the street name, to account for values like 'County Rd 126' which should be 'Country Road 126'. I also included some typos that I noticed while auditing the data.

In [2]:
expected = ["Street", "Avenue", "Boulevard", "Broadway", "Drive", "Center", "Circle", "Commons", "Court", "Highway",
            "Place", "Square", "Lane", "Loop", "Mall", "Road", "Trail", "Parkway", "Place", "Point", "Ramp", "Way",
            "Terrace", "Plaza", "Crescent"]

mapping = { "St": "Street", "st": "Street", "Strret": "Street", "STreet": "Street",
            "Ave": "Avenue", "ave": "Avenue", "Av": "Avenue",
            "Blvd": 'Boulevard',
            "Ct": "Court", "ct": "Court",
            "Cir": "Circle", "circle": "Circle",
            "Dr": "Drive", "drive": "Drive",
            "Hwy": "Highway",
            "Pkwy": "Parkway", "Pky": "Parkway",
            "Pl": "Place",
            "Rd": "Road", "rd": "Road", "Raod": "Road",
            "SH": 'State Highway',
            "SR": "State Road",
            "E": "East", "W": "West", "S": "South", "N": "North",
            "Main": "Main Street", "Mainstreet": "Main Street",
            "US": "US Highway",
            "Co": "Colorado", "CO": "Colorado",
            "lane": "Lane", "Ln": "Lane"
            }

I also manually corrected for some errors that I knew about from living in the area. For example, I know that Broadway does not have a street type like Avenue or Boulevard, that Baseline should be Baseline Road, and that East Colfax should be East Colfax Avenue. I confirmed all of these using Google Maps, just in case. 

I also noticed several street names included suite or unit numbers, which I removed.

The cleaning function ended up like this:

In [3]:
def update_st_name(name, mapping):

    name = name.replace('.', '')
    name = name.replace(',', '')
    name = name.replace('-', ' ')

    if 'Baselin' in name:
        return 'Baseline Road'

    if name == 'East Colfax':
        return 'East Colfax Avenue'

    name = name.split(' ')
    for i, name_word in enumerate(name):

        if any(unit == name_word for unit in ['Suite', 'suite', 'ste', 'Ste', 'Suit', 'Unit', 'unit']) \
                or '#' in name_word:
            name = name[:i]

        if name_word in mapping.keys():
            # print name
            name[i] = mapping[name_word]

    name = ' '.join(name)

    return name

There are still several problems remaining in the data. There are several street names without the street type (e.g. 'Lincoln') which could be referring to a number of actual streets. These would need to be fixed by comparing the location with a trusted source. Otherwise, most of the problems seem to stem from the data being entered into the wrong field. For example, there are several four-digit numbers, which are most likely address numbers. There are also several full addresses in this field. These can be fixed on a case-by-case basis for this particular dataset, but should be done programmatically for larger ones.

The next field I looked at was zipcodes. This was considerably simpler, as most of the data was entered correctly. I simply wrote a function that would check if a zipcode was valid (5-digit number between 80000 and 80700). To get everything to match, I had to convert some from the 9-digit version and remove the state abbreviation from a few others. This left a few values that were in the wrong field, as in the street name case. I also noticed three 6 digit numbers that looked like valid zipcodes with an extra digit (802377, 801111 and 801112). Although it is difficult to know which digit should be removed, I simply used the first 5 digits. 

I would suggest checking all the zipcodes against their latitude and longitude after the data is in the database. This would correct any human error in the inputting process, as well as correct my error if this assumption was wrong. However, this information may not be readily available or accessible. Ideally, there would be a free service with worldwide zipcode information including what latitude and longitude coordinates they covered. With a little searching, I have found http://federalgovernmentzipcodes.us/, which seems to do the trick for United States zipcodes, and there seem to be other country-specific ones. The above link notes that their data is accurate to two decimal places of latitude and longitude, which means a margin of error of around half a mile. Since this data is on the level of individual addresses, this could pose some problems on the borders of zipcodes.

The cleaning functions looked like this:

In [4]:
def is_valid_zip(zip):
    return len(zip) == 5 and str.isdigit(zip) and int(zip) in range(80000, 80700)


def update_zip(zip):

    if '-' in zip:
        zip = zip.split('-')
        zip = zip[0]
    elif ' ' in zip:
        zip = zip.split(' ')
        zip = zip[-1]
    elif 'CO' in zip:   # for format 'CO88888'
        zip = zip[2:]

    if is_valid_zip(zip):
        return zip
    elif is_valid_zip(zip[:-1]):
        return zip[:-1]
    else:
        return 'NULL'

Finally, I audited the city, county, state, and country fields with the following function:

In [5]:
def audit(osmfile, field):
    osm_file = open(osmfile, "r")
    count = {}

    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == 'node' or elem.tag == 'way':
            for tag in elem.iter('tag'):
                if tag.attrib['k'] == field:
                    v = tag.attrib['v']
                    if v in count.keys():
                        count[v] += 1
                    else:
                        count[v] = 1

The county and country audits showed that all values were in the same format, so nothing to do there.

For state, there were a few different formats, but everything was in the state of Colorado, so I simply changed everything to that.

I noticed a few problems in the city field. Several values, included the state, so I removed that. I also made a mapping of some typos that I noticed in the audit to the correct city names. Finally, I capitalized the first letter of each field.

Here is the code:

In [6]:
def update_state(state):
    return 'Colorado'


city_typos = {'Auroraa': 'Aurora',
              'CONIFER': 'Conifer',
              'Centenn': 'Centennial',
              'Dener': 'Denver',
              'ENGLEWOOD': 'Englewood',
              'Edgwater': 'Edgewater',
              'Hemderson': 'Henderson',
              'Littleton co': 'Littleton',
              'PARKER': 'Parker',
              'Thorton': 'Thornton',
              'Westminister': 'Westminster',
              'WestminsterO': 'Westminster',
              '+' : 'NULL',
              'CO': 'NULL',
              'CO 80129' : 'NULL'}

def update_city(city):
    city = city[0].upper() + city[1:]
    if ', CO' in city:
        city = city.split(',')[0]
    if city in city_typos.keys():
        city = city_typos[city]
    return city

To my knowledge, there are no more errors in these fields. However, they are missing from many of the entries, so a further step may be to add the correct values based on the latitude and longitude data. This can be done once the data is in the SQL database.

## Import to SQL Database

I used the schema and modified code from the class exercise to convert the XML data into python dictionaries, while applying my cleaning functions along the way.

In [9]:
import re

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'way':
        for field in way_attr_fields:
            way_attribs[field] = element.attrib[field]

        pos_counter = 0
        for node in element.iter('nd'):
            node_dict = {}

            node_dict['id'] = element.attrib['id']
            node_dict['node_id'] = node.attrib['ref']
            node_dict['position'] = pos_counter

            pos_counter += 1
            way_nodes.append(node_dict)

    elif element.tag == 'node':
        for field in node_attr_fields:
            node_attribs[field] = element.attrib[field]

    for tag in element.iter('tag'):
        if not problem_chars.search(tag.attrib['k']):
            tag_dict = {}
            tag_dict['id'] = element.attrib['id']

            #change fields that I cleaned
            if tag.attrib['k'] == 'addr:street':
                tag_dict['value'] = update_st_name(tag.attrib['v'], mapping)
            elif tag.attrib['k'] == 'addr:state':
                tag_dict['value'] = update_state(tag.attrib['v'])
            elif tag.attrib['k'] == 'addr:postcode':
                tag_dict['value'] = update_zip(tag.attrib['v'])
            elif tag.attrib['k'] == 'addr:city':
                tag_dict['value'] = update_city(tag.attrib['v'])
            else:
                tag_dict['value'] = tag.attrib['v']

            if ':' not in tag.attrib['k']:
                tag_dict['key'] = tag.attrib['k']
                tag_dict['type'] = default_tag_type
            else:
                k = tag.attrib['k'].split(':')
                tag_dict['type'] = k[0]
                tag_dict['key'] = ':'.join(k[1:])

            tags.append(tag_dict)

    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

The resulting csv files were then imported into a sqlite database for the following queries.

## Exploring the Database

### File Sizes

denver-boulder_colorado.osm - 825.4 MB   
osm_denver_boulder.db - 58.9 MB   
nodes.csv - 31.9 MB   
nodes_tags.csv - 1.1 MB    
ways.csv - 2.6 MB    
ways_nodes.csv - 10.3 MB   
ways_tags.csv - 6.1 MB   

### Number of Unique Users

In [15]:
QUERY = 'SELECT COUNT(DISTINCT(s.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) as s'

1334

### Number of Nodes

In [14]:
QUERY = 'SELECT COUNT(*) FROM nodes'

372635

### Number of Ways

In [12]:
QUERY = 'SELECT COUNT(*) FROM ways'

41694

### Top Ten Amenities

In [16]:
QUERY = 'SELECT value, COUNT(*) as n FROM node_tags WHERE key = "amenity" GROUP BY value ORDER BY n DESC LIMIT 10'

* restaurant 181 
* fast_food 90
* bicycle_parking 76
* bench 72
* place_of_worship 69
* school 66
* fuel 59
* parking 47
* cafe 34
* toilets 34

### Top Ten Restaurant Types

In [13]:
QUERY = 'SELECT value, COUNT(*) as n FROM node_tags WHERE key = "cuisine" GROUP BY value ORDER BY n DESC LIMIT 10'

* sandwich 25
* mexican 22
* pizza 20
* burger 19
* american 18
* coffee_shop 13
* asian 8
* ice_cream 5
* italian 5
* chinese 4

### Top Five Religious Denominations

In [18]:
QUERY = 'SELECT value, COUNT(*) as n FROM node_tags WHERE key = "denomination" GROUP BY value ORDER BY n DESC LIMIT 5'

* lutheran 10
* methodist 7
* baptist 5
* catholic 4
* episcopal 3

### Are there more coffee shops or marijuana dispensaries in the Denver area?

In [19]:
QUERY = 'SELECT COUNT(*) FROM node_tags WHERE (key = "amenity" AND value = "cafe") \
OR (key = "cuisine" AND value = "coffee_shop")'

Number of cafes/coffee shops: 47

In [20]:
QUERY = 'SELECT COUNT(*) FROM node_tags WHERE key = "amenity" AND value = "dispensary"'

Number of dispensaries: 0 (clearly, the data is incomplete)

## Conclusions

Although there is still plenty to be done with this dataset, I have standardized the addresses to make them significantly more readable and correct. Similar cleaning could be done to make the categorizations more useful (for example, amenity-restaurant-coffee shop and amenity-cafe are the same thing), and it would be good to verify the correctness of the data by checking the latitude and longitude against a trusted source and confirming that the full address information is accurate. From there, what remains is to add more data to get a complete map of the area.

## Sources

https://discussions.udacity.com/t/creating-db-file-from-csv-files-with-non-ascii-unicode-characters/174958/7  
https://discussions.udacity.com/t/last-quiz-preparing-for-database-in-lesson-6-mongodb/44559/30  
http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html#connecting-to-an-sqlite-database  